In [1]:
from tg_parser import TelegramParser

async def example():
    # Использование с контекстным менеджером
    async with TelegramParser() as parser:
        # Получение сообщений из канала
        url = 'https://t.me/s/seeallochnaya'
        messages = await parser.get_channel_messages(url, limit=5, reverse=True)
        return messages


# Запуск примера
ss = TelegramParser.run_async(example())

In [2]:
for i in range( len(ss)):
    ss[i].message = 'Лекция по машинному обучению 20 января в 18:00, аудитория 101'

In [3]:
ss[3].message

'Лекция по машинному обучению 20 января в 18:00, аудитория 101'

In [4]:
import os
import dotenv
from openai import OpenAI
from event_miner_agent import EventMinerAgent

# Загружаем переменные окружения
dotenv.load_dotenv()

# Инициализация LLM
llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Создание агента для извлечения событий
agent = EventMinerAgent(llm)

# Получаем сообщения из канала (используем ss из предыдущей ячейки)
try:
    # Обрабатываем сообщения типа Message из telethon
    events = agent.process_messages(ss)
    
    print(f"Найдено событий: {len(events)}\n")
    for i, event in enumerate(events, 1):
        print(f"Событие {i}:")
        print(f"  Название: {event.title}")
        print(f"  Тип: {event.event_type}")
        print(f"  Онлайн: {event.is_online}")
        print(f"  Место: {event.location}")
        print(f"  Дата: {event.date}")
        print(f"  Время: {event.time}")
        print(f"  ID сообщения: {event.source_message_id}")
        print()
except NameError:
    print("Сначала выполните ячейку с получением сообщений (ss)")
except Exception as e:
    print(f"Ошибка при обработке: {e}")


Найдено событий: 5

Событие 1:
  Название: Лекция по машинному обучению
  Тип: лекция
  Онлайн: False
  Место: аудитория 101
  Дата: 2026-01-20
  Время: 18:00
  ID сообщения: 3167

Событие 2:
  Название: Лекция по машинному обучению
  Тип: лекция
  Онлайн: False
  Место: аудитория 101
  Дата: 2026-01-20
  Время: 18:00
  ID сообщения: 3168

Событие 3:
  Название: Лекция по машинному обучению
  Тип: лекция
  Онлайн: False
  Место: аудитория 101
  Дата: 2026-01-20
  Время: 18:00
  ID сообщения: 3169

Событие 4:
  Название: Лекция по машинному обучению
  Тип: лекция
  Онлайн: False
  Место: аудитория 101
  Дата: 2026-01-20
  Время: 18:00
  ID сообщения: 3170

Событие 5:
  Название: Лекция по машинному обучению
  Тип: лекция
  Онлайн: False
  Место: аудитория 101
  Дата: 2026-01-20
  Время: 18:00
  ID сообщения: 3171



In [ ]:
# Пример работы с сообщениями из избранного
async def extract_events_from_saved():
    async with TelegramParser() as parser:
        # Получаем сообщения из избранного
        saved_messages = await parser.get_saved_messages(limit=20, text_only=True, reverse=True)
        
        if saved_messages:
            # Обрабатываем сообщения типа Message из telethon
            events = agent.process_messages(saved_messages)
            
            print(f"Найдено событий в избранном: {len(events)}\n")
            for i, event in enumerate(events, 1):
                print(f"Событие {i}:")
                print(f"  Название: {event.title}")
                print(f"  Тип: {event.event_type}")
                print(f"  Онлайн: {event.is_online}")
                print(f"  Место: {event.location}")
                print(f"  Дата: {event.date}")
                print(f"  Время: {event.time}")
                if event.original_text:
                    print(f"  Оригинал: {event.original_text[:100]}...")
                print()
            
            return events
        else:
            print("Нет сообщений в избранном")
            return []

# Запуск
events_from_saved = TelegramParser.run_async(extract_events_from_saved())


In [ ]:
# Пример работы с батчами (для большого количества сообщений)
async def extract_events_batch():
    async with TelegramParser() as parser:
        # Получаем много сообщений
        saved_messages = await parser.get_saved_messages(limit=100, text_only=True, reverse=True)
        
        if saved_messages:
            # Обрабатываем батчами по 10 сообщений
            events = agent.process_messages_batch(saved_messages, batch_size=10)
            
            print(f"Всего обработано сообщений: {len(saved_messages)}")
            print(f"Найдено событий: {len(events)}\n")
            
            # Группируем по типу события
            events_by_type = {}
            for event in events:
                event_type = event.event_type or "неизвестно"
                if event_type not in events_by_type:
                    events_by_type[event_type] = []
                events_by_type[event_type].append(event)
            
            print("События по типам:")
            for event_type, type_events in events_by_type.items():
                print(f"  {event_type}: {len(type_events)}")
            
            return events
        else:
            print("Нет сообщений")
            return []

# Запуск (раскомментируйте для выполнения)
# events_batch = TelegramParser.run_async(extract_events_batch())


In [2]:
saved_messages

NameError: name 'saved_messages' is not defined